In [1]:
%run package_import.ipynb
%run data_profiling.ipynb

In [2]:
from sqlalchemy import create_engine, types
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/kaggle_hour_price', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

In [59]:
df_combined = pd.read_sql_query(

''' 
select *,
    'train' as source
from raw_train

union all

select *, 
    cast(null as float) as SalePrice,
    'test' as source
from raw_test

'''
,dbConnection)

In [60]:
df_profile = main_data_profiling(df_combined)

In [50]:
df_profile[(df_profile['null_values']>0) & (df_profile['data_type']=='object')].sort_values(by='null_values',ascending=False)

,column_names,col_desc,value_desc,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
72,PoolQC,Pool quality,"{'Ex': 'Excellent', 'Gd': 'Good', 'TA': 'Average/Typical', 'Fa': 'Fair', 'NA': 'No Pool'}",object,4,NaN,NaN,NaN,NaN,2909,2919,0.996574,NaN,NaN
74,MiscFeature,Miscellaneous feature not covered in other categories,"{'Elev': 'Elevator', 'Gar2': '2nd Garage (if not described in garage section)', 'Othr': 'Other', 'Shed': 'Shed (over 100 SF)', 'TenC': 'Tennis Court', 'NA': 'None'}",object,5,NaN,NaN,NaN,NaN,2814,2919,0.964029,NaN,NaN
6,Alley,Type of alley access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved', 'NA': 'No alley access'}",object,3,NaN,NaN,NaN,NaN,2721,2919,0.932169,NaN,NaN
73,Fence,Fence quality,"{'GdPrv': 'Good Privacy', 'MnPrv': 'Minimum Privacy', 'GdWo': 'Good Wood', 'MnWw': 'Minimum Wood/Wire', 'NA': 'No Fence'}",object,5,NaN,NaN,NaN,NaN,2348,2919,0.804385,NaN,NaN
57,FireplaceQu,Fireplace quality,"{'Ex': 'Excellent - Exceptional Masonry Fireplace', 'Gd': 'Good - Masonry Fireplace in main level', 'TA': 'Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement', 'Fa': 'Fair - Prefabricated Fireplace in basement', 'Po': 'Poor - Ben Franklin Stove', 'NA': 'No Fireplace'}",object,6,NaN,NaN,NaN,NaN,1420,2919,0.486468,NaN,NaN
63,GarageQual,Garage quality,"{'Ex': 'Excellent', 'Gd': 'Good', 'TA': 'Typical/Average', 'Fa': 'Fair', 'Po': 'Poor', 'NA': 'No Garage'}",object,6,NaN,NaN,NaN,NaN,159,2919,0.054471,NaN,NaN
60,GarageFinish,Interior finish of the garage,"{'Fin': 'Finished', 'RFn': 'Rough Finished', 'Unf': 'Unfinished', 'NA': 'No Garage'}",object,4,NaN,NaN,NaN,NaN,159,2919,0.054471,NaN,NaN
64,GarageCond,Garage condition,"{'Ex': 'Excellent', 'Gd': 'Good', 'TA': 'Typical/Average', 'Fa': 'Fair', 'Po': 'Poor', 'NA': 'No Garage'}",object,6,NaN,NaN,NaN,NaN,159,2919,0.054471,NaN,NaN
58,GarageType,Garage location,"{'2Types': 'More than one type of garage', 'Attchd': 'Attached to home', 'Basment': 'Basement Garage', 'BuiltIn': 'Built-In (Garage part of house - typically has room above garage)', 'CarPort': 'Car Port', 'Detchd': 'Detached from home', 'NA': 'No Garage'}",object,7,NaN,NaN,NaN,NaN,157,2919,0.053786,NaN,NaN
31,BsmtCond,Evaluates the general condition of the basement,"{'Ex': 'Excellent', 'Gd': 'Good', 'TA': 'Typical - slight dampness allowed', 'Fa': 'Fair - dampness or some cracking or settling', 'Po': 'Poor - Severe cracking, settling, or wetness', 'NA': 'No Basement'}",object,5,NaN,NaN,NaN,NaN,82,2919,0.028092,NaN,NaN


### ETL Process

In [61]:
# The loop is to convert description to jason to fill na value
# initalise the dictionary
desc_dict, attr_dict, col_desc = {}, {}, {}
with open('House_Prices/data_description.txt', encoding='utf8') as f:
    for line in f:
        # convert line into list
        words = line.split()
        # if the line contains ":", the line includes a column
        # and its description
        if words and ':' in line:
            # convert the line into list, recognise the first item from the list
            # as the column and remove ":"
            col = words[0].replace(':','')
            # save the column as a key into description dictionary
            desc_dict[col] = {}
            col_desc[col] = ' '.join(words[1:])
        # if the line is blank, move to the next one
        elif  not words:
            continue
        else:
            # the attr_desc is to contain values for each column
            # concatenate items except the value from the list to 
            # rebuild the description of each value
            attr_desc = ' '.join(words[1:])
            # store the first item of the list as the value
            attr = words[0]
            # save the the attribute dictionary as sub-dictionary to
            # description description
            desc_dict[col][attr] = attr_desc

In [62]:
df_profile = main_data_profiling(df_combined)
df_profile.insert(1, 'col_desc', np.nan)
df_profile.insert(2, 'value_desc', np.nan)
df_profile['col_desc'] = df_profile['column_names'].map(col_desc)
df_profile['value_desc'] = df_profile['column_names'].map(desc_dict)
df_profile['value_desc'] = df_profile['value_desc'].astype('str')

#### 1. Replace NA

In [66]:
df_combined['SaleType'] = df_combined['SaleType'].fillna('WD')
df_combined['MSZoning'] = df_combined['MSZoning'].fillna('RL')
df_combined['Functional'] = df_combined['Functional'].fillna('Typ')
df_combined['Utilities'] = df_combined['Utilities'].fillna('AllPub')
df_combined['Electrical'] = df_combined['Electrical'].fillna('SBrkr')
df_combined['KitchenQual'] = df_combined['KitchenQual'].fillna('TA')
df_combined['Exterior2nd'] = df_combined['Exterior2nd'].fillna('VinylSd')
df_combined['Exterior1st'] = df_combined['Exterior1st'].fillna('VinylSd')

# the loop is to fill na value based on descriptioin automatically
for col in df_combined.columns:
    # if a column has null value and its value includes "NA"
    # replace NA with the value mentioned in the description;
    # otherwise, skip it. 
    if df_profile[df_profile['column_names'] == col]['null_values'].squeeze() > 0:
        if df_profile[df_profile['column_names'] == col]['data_type'].squeeze() == 'object':
                    df_combined[col] = df_combined[col].fillna('No')
        elif df_profile[df_profile['column_names'] == col]['data_type'].squeeze() == 'float64':
            df_combined[col] = df_combined[col].fillna(0)

#### 2. Check duplication

In [127]:
df_combined[df_combined.duplicated()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,source


#### map rates for quality

### Upload transformed data

In [141]:
df_combined.to_sql('transformed_combined', dbConnection, if_exists='replace', index=False)

In [32]:
df_profile.to_sql('raw_data_profile', dbConnection, if_exists='replace', index=False, method='multi')